In [4]:
import numpy as np
from copy import deepcopy

MIN_SIZE = 1
MAX_SIZE = 50
MIN_CYCLE = 300
MAX_CYCLE = 1000
MIN_DDL = 0.1
MAX_DDL = 1
MIN_RESOURCE = 0.4
MAX_RESOURCE = 1.5
MIN_POWER = 1
MAX_POWER = 24
CAPABILITY_E = 4
K_ENERGY_LOCAL = 5 * 1e-27

MIN_ENE =0.5
MAX_ENE = 3.2
HARVEST_RATE = 0.001
W_BANDWIDTH = 40

K_CHANNEL = 10
S_E = 400
N_UNITS = 8
MAX_STEPS = 10



In [ ]:
class uavENV():

    

    def __init__(self, n_agents):
        
        self.state_size = 7
        self.action_size = 3
        self.n_agents = n_agents
        self.W_BANDWIDTH = W_BANDWIDTH

        self.S_power = np.zeros(self.n_agents)
        self.initial_energy = np.zeros(self.n_agents)
        self.S_energy = np.zeros(self.n_agents)
        self.S_gain = np.zeros(self.n_agents)
        self.S_size = np.zeros(self.n_agents)
        self.S_cycle = np.zeros(self.n_agents)  
        self.S_ddl = np.zeros(self.n_agents)
        self.S_res = np.zeros(self.n_agents)
        self.action_lower_bound = [0,  0.01, 0.01] 
        self.action_higher_bound = [1, 1, 1]
        for n in range(self.n_agents):
            self.S_size[n] = np.random.uniform(MIN_SIZE, MAX_SIZE)
            self.S_cycle[n] = np.random.uniform(MIN_CYCLE, MAX_CYCLE)
            self.S_ddl[n] = np.random.uniform(MIN_DDL, MAX_DDL)
            self.S_res[n] = np.random.uniform(MIN_RESOURCE, MAX_RESOURCE)

    
    
    def reset(self):
        self.step = 0
        for n in range(self.n_agents):
            self.S_size[n] = np.random.uniform(MIN_SIZE, MAX_SIZE)
            self.S_cycle[n] = np.random.uniform(MIN_CYCLE, MAX_CYCLE)
            self.S_ddl[n] = np.random.uniform(MIN_DDL, MAX_DDL)
            self.S_energy[n] = deepcopy(self.initial_energy[n])
        self.S_energy = np.clip(self.S_energy, MIN_ENE, MAX_ENE)
        self.state = np.array([self.S_power[n], self.S_gain[n], self.S_energy[n], self.S_size[n], self.S_cycle[n], self.S_ddl[n], self.S_res[n] for n in range(self.n_agents)])
        return self.state
    
    
    
    def step(self, action):
        A_decision = np.zeros(self.n_agents)
        A_res = np.zeros(self.n_agents)
        A_power = np.zeros(self.n_agents)
        for n in range(self.n_agents):
            A_decision[n] = action[n][0]
            A_res[n] = action[n][1] * self.S_res[n] * 10 ** 9
            A_power[n] = action[n][2] * 10 ** ((self.S_power[n]-30)/10)
        x_n = A_decision

        DataRate = self.W_BANDWIDTH * 10 ** 6  * np.log(1 + A_power * 10 **(self.S_gain/10)) / np.log(2)
        DataRate = DataRate / K_CHANNEL
        Time_proc = self.S_size*8*1024*self.S_cycle / (CAPABILITY_E*10**9)
        Time_local = self.S_size*8*1024*self.S_cycle / A_res
        Time_max_local = self.S_size*8*1024*self.S_cycle / (MIN_RESOURCE*10**9)
        Time_off = self.S_size*8*1024/ DataRate
        for i in range(self.n_agents):
            if x_n[i] == 2:
                Time_off[i] = MAX_DDL
                x_n[i] = 1
        Time_finish = np.zeros(self.n_agents)

        


    



In [50]:
env=uavENV()
print(env)


[10  7  5 10 11 48 20 65 92 49 30 13 57 80]


In [9]:
UAV_energy_list = np.random.randint(0, 101, 2)
print(UAV_energy_list)

print(UAV_energy_list[1])


[13 96]
96
